In [ ]:
# TITANIC SURVIVAL PREDICTION 

#### DS INTERN Marry-Anne Learning features

Lets Get In Detail

Its going to be a lot

https://www.kaggle.com/datasets/yasserh/titanic-dataset/data

In [ ]:
# import the libraries 
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

## OPEN AND VIEW THE DATA 

In [ ]:
df=pd.read_csv(r"D:\datasets kaggle\Titanic-Dataset.csv")
df.head(5)

detials of the features:
    
    PassengerId: Passenger ID
    Survived: Weather Survived or not: 0 = No, 1 = Yes
    Pclass - Ticket class: 1 = 1st, 2 = 2nd, 3 = 3rd (1st ~ Upper; 2nd ~ Middle; 3rd ~ Lower)
    Name:Name of the Passenger
    Sex: Gender
    Age:Age in Years    
    SibSp: No. of siblings / spouses aboard 
        the Titanic  Sibling: Brother, Sister, Stepbrother, or Stepsister of Passenger Aboard Titanic
        Spouse: Husband or Wife of Passenger Aboard Titanic (Mistresses and Fiances Ignored)

    Parch:No. of parents / children aboard the Titanic    
        Parent: Mother or Father of Passenger Aboard Titanic
        Child: Son, Daughter, Stepson, or Stepdaughter of Passenger Aboard Titanic
        Some children travelled only with a nanny, therefore parch=0 for them.
    
    Ticket:Ticket number
    Fare:Passenger fare
    cabin:cabin 
    embarked Port of Embarkation (C = Cherbourg; Q = Queenstown; S = Southampton)    

In [ ]:
#collecting information on the data 
df.info()

In [ ]:
#an explanation of the data  
df.describe()

the age has missing values 
p class, Sibsp, parch, fare are skewed. 

In [ ]:
df.describe(include=object)

the cabbin has missing values 
embarked has missing values 

next is the...

# PREPROCESSING OF DATA

##### *After obtaining the person's name and title, which are both unique, the passenger ID columns are being dropped. 

In [ ]:
# looking up titles in the name field 
df['Name'].str.split(" ", expand=True)[1].unique()

        1."Mlle." is an abbreviation for "Mademoiselle," a French courtesy title used to address young unmarried women or girls.
        2.The title "Countess" is a noble title in various European countries and is used to address the wife or widow of a Count, an Earl, or an equivalent noble rank. 
        3.In Dutch and Belgian nobility, "Jonkheer" (pronounced yon-kuhr) is an honorific that translates to "young lord" or "young gentleman" in English. I
        4."Mme." is an abbreviation for "Madame," a French courtesy title that is the equivalent of "Mrs." or "Ma'am" in English. 
        5.Miss: "Miss" is used as a title before the name of an unmarried woman or girl.
        6.Master: "Master" is used as a title before the name of a young boy. It is used for boys who have not yet reached adulthood. 
        7.Mr.: Used before a man's name to address him as "Mister.
        8.Mrs.: Used before a married woman's name. 

In [ ]:
import re

In [ ]:
# obtaining the name column's title pattern extraction
title_pattern = r'(Mr\.|Mrs\.|Miss\.|Master\.|Don\.|Rev\.|Dr\.|Ms\.|Major\.|Col\.|Capt\.| Lady\.|Mme\.|Sir\.|Mlle\.|Jonkheer\.)'

# Make a new column called "Title" and use regular expressions to extract titles.
df['Title'] = df['Name'].str.extract(title_pattern, expand=False)


In [ ]:
# checking for null values in title column 
df[df['Title'].isnull()]

In [ ]:
# Because the title appears in the third column following the split, one column contains a null value. therefore manually filling it
df['Title']=df['Title'].fillna('Countess.')

In [ ]:
# dropping passenger id and name of the passenger 
df1=df.drop(['PassengerId','Name'], axis=1)

### *looking for any missing values

In [ ]:
# looking for missing values in the columns 
((df1.isnull().sum())/df1.shape[0])*100

The Age field has 19 % missing values, whereas the Cabin column has 77 percentage missing values.

#### * treating Age column based on the title 

In [ ]:
# checking for unique values in age
df1['Age'].unique()

In [ ]:
# taking average of age according to title as each title points out to a particular group 
df1.groupby('Title')['Age'].median()

In [ ]:
# imputing the median age of each titke to the missing values on age column 
df1['Age']=df1['Age'].fillna(df1.groupby('Title')['Age'].transform('median'))

#### *Treating Cabin column

In [ ]:
# checking for columns with cabin values. 
# Cabin may not be available to all passengers. 
df1[~(df1['Cabin'].isnull())]

In [ ]:
# cabin has an alphabet which represent the deck and a number followed by which is the room number. 
# slicing the cabin and storing the deck as a new column deck. 
df1['Deck']=df1['Cabin'].str.slice(0,1)

In [ ]:
# filling the null values in the cabin column 
df1['Cabin']=df1['Cabin'].fillna('not available')

In [ ]:
# filling null values in deck with not available 
df1['Deck']=df1['Deck'].fillna('not available')

In [ ]:
# dropping cabin column 
df2=df1.drop('Cabin', axis=1)

##### * creating a new column named all possible decks based on passenger class. 

In [ ]:
A Deck ---- exclusively for First Class 
B Deck ---- First Class cabins
C Deck ---- First Class cabins
D Deck ---- First, Second and Third Class passengers had cabins
E Deck ---- passenger accommodation for all three classes
F Deck ---- Second and Third Class passengers
G Deck ---- orlop (partial) decks  

https://en.wikipedia.org/wiki/Titanic#Background
    The boat deck, on which the lifeboats were housed. It was from here during the early hours of 15 April 1912 that Titanic's lifeboats were lowered into the North Atlantic. The bridge and wheelhouse were at the forward end, in front of the captain's and officers' quarters. The bridge stood 8 feet (2.4 m) above the deck, extending out to either side so that the ship could be controlled while docking. The wheelhouse stood within the bridge. The entrance to the First Class Grand Staircase and gymnasium were located midships along with the raised roof of the First Class lounge, while at the rear of the deck were the roof of the First Class smoke room and the relatively modest Second Class entrance. The wood-covered deck was divided into four segregated promenades: for officers, First Class passengers, engineers, and Second Class passengers respectively. Lifeboats lined the side of the deck except in the First Class area, where there was a gap so that the view would not be spoiled.[18][19]

The promenade deck, sometimes known as a deck, ran the full 546 feet (166 m) of the superstructure. It included First Class cabins, the First Class lounge, a smoking room, reading and writing rooms, and Palm Court, all of which were designated for use by First Class guests alone.[18]
   
       The highest level of the hull and the top weight-bearing deck was the bridge deck, or B Deck. Here were additional First Class guest accommodations, including six magnificent staterooms (cabins) with separate private promenades. The à la carte restaurant and the Café Parisien on the Titanic offered First Class guests opulent dining experiences. Both were operated by chefs under contract and their employees, all of whom perished in the accident. The entry hall and Second Class smoking room were situated on this deck. The ship's elevated forecastle, which housed the anchor housings, various pieces of equipment, and Number 1 hatch (the main door leading to the cargo holds), was located ahead of the bridge deck.[c] Third Class passengers utilized the elevated poop deck, which was 32 meters (106 feet) long, as a promenade, situated aft of the bridge deck. As the Titanic sunk, many of its passengers and crew made their last stand there. Well decks kept the forecastle and poop deck apart from the bridge deck.[20][21]
    
    The uppermost deck that ran straight from bow to stern was the shelter deck, or C Deck. It included both well decks; the Third Class promenade used the forward deck. Below, there were crew cabins.Beneath the poop deck were the Third Class public rooms and the forecastle. The Second Class library and most First Class cabins were located in between.[20][22]
    
    The First Class dining saloon, First Class reception room, and Second Class dining saloon were the three huge public spaces that dominated D Deck, the saloon deck. Passengers in third class had an open space. This deck included accommodations for First, Second, and Third Class passengers, with fireman's sleepers situated in the bow. Strictly speaking, only eight of the fifteen watertight bulkheads on the ship reached this greatest level.[20][23]
    
    The top deck, or E Deck, had berths for chefs, seamen, stewards, and trimmers in addition to passenger accommodations for all three classes. Along with its length was a lengthy corridor known as "Scotland Road," after a well-known Liverpool street. Both crew personnel and passengers traveling in third class utilized Scotland Road.[20][24]
    
    The middle deck, or F Deck, was the final fully finished deck and was mostly used by departments of the crew as well as guests traveling in Second and Third Class. This was the location of the Turkish bath, swimming pool, and kennels in addition to the Third Class dining saloon.[20][24][25]
    
    Located barely over the waterline, G Deck was the lowest passenger deck in its whole and had the lowest portholes. This was the location of the squash court as well as the traveling post office, where packages and letters were processed and ready for distribution upon ship docking. This area was also used to store food. Something interrupted the deck.by orlop (partial) decks over the engine, turbine, and boiler rooms at various locations.[20][26]
    On the lowest level of the ship, below the waterline, were the orlop decks and the tank top below that. The tank top, or innermost part of the ship's hull, served as a platform for the boilers, engines, turbines, and electrical generators of the vessel, while the orlop decks were utilized as cargo areas. Passengers would not have been allowed to witness the engine and boiler rooms located in this section of the ship. There were flights of steps connecting them to the higher decks of the ship; twin spiral stairways in the bow led up to D Deck.[20][26]


In [ ]:
# creating the new column called all possible decks. 

conditions = [
    (df2['Pclass'] == 1) & (df2['Deck'] == 'not available'),
    ((df2['Deck'] == 'A') | (df2['Deck'] == 'B') | (df2['Deck'] == 'C') | (df2['Deck'] == 'D')),
    (df2['Pclass'] == 2) & (df2['Deck'] == 'not available'),
    ((df2['Deck'] == 'D') | (df2['Deck'] == 'E') | (df2['Deck'] == 'F')),
    (df2['Pclass'] == 3) & (df2['Deck'] == 'not available'),
    ((df2['Deck'] == 'D') | (df2['Deck'] == 'E') | (df2['Deck'] == 'F') | (df2['Deck'] == 'G'))]

choices = ['A/B/C/D', 'A/B/C/D', 'D/E/F', 'D/E/F', 'D/E/F/G', 'D/E/F/G']

df2['all possible Decks'] = np.select(conditions, choices, default=df2['Deck'])


### *Adding a new column showing the number of people in that particular ticket and the cost per person 

In [ ]:
# getting a new column with no of persons in that specified ticket 
value=df2['Ticket'].value_counts()
df2['persons_in_ticket']=df2['Ticket'].map(value)

In [ ]:
value

In [ ]:
# creating fare per person using persond per ticket 
df2['fare per person']=df2['Fare']/df2['persons_in_ticket']

In [ ]:
# checking unique values in the column
df2['persons_in_ticket'].unique()

##### *create a travel category column 

In [ ]:
# creating a new column called traveler actegory -- big family(5), small family(5), nuclear family(4,3), bi(2), solo(1), 

In [ ]:
# creating a new user defined function to apply on persond in ticket 
def travel(x):
    if x==1:
        return 'solo'
    elif x==2:
        return 'bi'
    elif (x==3) | (x==4):
        return 'small family'
    elif (x==5) | (x==6) |(x==7):
        return 'large family'
    
    
df2['travelercategory']=df2['persons_in_ticket'].apply(travel)

#### * CREATING A NEW AGE GROUP COLUMN 

In [ ]:
## Age group columns 

In [ ]:
df2['age group']=df1['Age'].apply(lambda x: 'infant' if 0<x<=2 else 'child' if 2<x<=16 \
                                  else 'youngadults' if 16<x<=30 else \
                                 'middleaged' if 30<x<=45 else 'oldadults' if 45<x<=60  \
                                  else 'seniorcitizens' if x>60  else 'notavailable' )

# VISUALIZATION OF THE DATA 

In [ ]:
df2_num=df2.select_dtypes(exclude=object)
df2_cat=df2.select_dtypes(include=object)


In [ ]:
df2_num

In [ ]:
plt.figure(figsize=(10,3))
re=1
for i in df2_num.columns:
    plt.subplot(2,4,re)
    sns.kdeplot(df2[i], color='black')
    plt.title(i)
    re+=1
    
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10,3))
re=1
for i in df2_num.columns:
    plt.subplot(2,4,re)
    sns.boxplot(df2[i], color='black')
    plt.title(i)
    re+=1
    
plt.tight_layout()
plt.show()

In [ ]:
The age, fare, persons per ticket, fare per person all has outliers. 

In [ ]:
df2['Pclass'].value_counts().plot(kind='bar', cmap='viridis')
plt.title('passenger count based on  Pclass')
plt.show()

third class has the highest number of passengers compare to other 2 classes 

In [ ]:
df2['Sex'].value_counts().plot(kind='bar', cmap='cividis')
plt.title('passenegr count based on Sex')
plt.show()

The number of male passengers is high compared to the female passengers. 

In [ ]:
plt.figure(figsize=(15,6))
a=df2['Ticket'].value_counts()
a[a>2].plot(kind='bar', cmap='icefire')
plt.title('persons per ticket')
plt.show()

different ticket has different counts, highest being 7 and lowest being 1.

In [ ]:
plt.figure(figsize=(15,6))
a=df2['Embarked'].value_counts()
a[a>2].plot(kind='bar', cmap='turbo')
plt.title('passenger count based on Embarked')
plt.show()

the passenger count is high for southamptin.

In [ ]:
plt.figure(figsize=(15,6))
a=df2['Title'].value_counts()
a[a>2].plot(kind='bar', cmap='summer')
plt.title('passenger count based on Title')
plt.show()

passenger with title mr. is higher in count followed by miss. 

In [ ]:
a=df2['age group'].value_counts()
a.index

In [ ]:
# creating  apie diagram to knoe the percentage of age group 
plt.figure(figsize=(6,8))
plt.pie(a, autopct='%1.1f%%',
        startangle=140)
plt.legend(a.index)
plt.title('Pie Chart for age group')
plt.show()

Young adults made up 52% of the passengers, followed by middle-aged people (24%), and senior individuals (2.5%).

In [ ]:
# pie chat to understand the percentage of people in each travel catgeory
b=df2['travelercategory'].value_counts()
plt.figure(figsize=(6,8))
plt.pie(b, autopct='%1.1f%%',
        startangle=140)
plt.legend(b.index)
plt.title('Pie Chart for travelercategory Data')
plt.show()

61 percentage of the total travelrs were solo travelers followed by bi-travelers. 

# Bivariate analysis

In [ ]:
sns.pairplot(df2)

In [ ]:
sns.heatmap(df2.corr(), annot=True, mask=np.triu(df2.corr()))
plt.show()

there is be a correlation between fare and passengerclass

# variable visualization using the target column. ('Survived')

### sex and survived

In [ ]:
sns.set_style("whitegrid")

sns.countplot(hue=df2['Survived'], x=(df2['Sex']),palette='bone' )
plt.title('count of passengers survival based on sex')
plt.show()

In [ ]:
df2[['Sex','Survived']].value_counts()

In [ ]:
df2[['Sex']].value_counts()

In [ ]:
df2['Sex'].count()

In [ ]:
# % of male survived = survived male passengers /total male passenger
t_s=((233+109)/891)*100
t_m_s=(109/891)*100
t_f_s=(233/891)*100
p_m=(109/577)*100
f_m=(233/314)*100
print('total survival percentage of passengers',t_s )
print('total male survival percentage in survived',t_m_s)
print('total female survival percentage in survived',t_f_s)
print('percentage of male survived', p_m)
print('percentage of females survived', f_m)

the survival percentage of females is high 

#### survived vs age group

In [ ]:
plt.figure(figsize=(10,6))
sns.countplot(x=df2['Survived'], hue=df2['age group'], palette='bone')

In [ ]:
count of youngadults who died is more. 

#### traveler category vs survived

In [ ]:
plt.figure(figsize=(10,6))
sns.countplot(x=df2['Survived'], hue=df2['travelercategory'], palette='bone')

### pclass vs survived 

In [ ]:
plt.figure(figsize=(10,6))
sns.countplot(x=df2['Survived'], hue=df2['Pclass'], palette='bone')

There are few third-class guests who survive. 

#### title vs survived

In [ ]:
plt.figure(figsize=(10,6))
sns.countplot(x=df2['Survived'], hue=df2['Title'])

In [ ]:
mr. title people died more 

##### fare per person vs survival category 

In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(x=df2['Survived'], y=df2['fare per person'])

In [ ]:
survived category has comparably high fare. 

In [ ]:
from gapminder import gapminder

In [ ]:
sns.catplot(data=df2, x="Pclass", y="Fare", hue="Survived", kind="box")

fare is high for survived passengers. 

## MODEL BUILDING 

In [ ]:
# import the libraries
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble  import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
from sklearn.model_selection import cross_val_score,KFold
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split,GridSearchCV
from scipy import stats
from sklearn.metrics import classification_report,roc_curve,confusion_matrix,\
accuracy_score,recall_score,precision_score,f1_score,cohen_kappa_score
from sklearn.metrics import log_loss
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import ConfusionMatrixDisplay, accuracy_score, confusion_matrix,accuracy_score,classification_report,recall_score,precision_score,f1_score,roc_auc_score

In [ ]:
# drop ticket and title 
df3=df2.drop(['Ticket','Title'], axis=1)

In [ ]:
df3

In [ ]:
# get dummies 
df4=pd.get_dummies(df3, drop_first=True)

In [ ]:
df4

In [ ]:
# create an empty dataframe to store the scores for various algorithms
perf_score = pd.DataFrame(columns=["Model", "Accuracy","Recall",
                                   "Precision","F1 Score",'reliability'] )

In [ ]:
def per_measures(model,test,pred):
    accuracy    = accuracy_score(test,pred)
    f1score     = f1_score(test,pred)
    recall      = recall_score(test,pred)
    precision   = precision_score(test,pred)
    reliability = cohen_kappa_score(test,pred)
    return (accuracy,recall,precision,f1score,reliability)

In [ ]:
def update_performance (name, model,test,pred):
    # assign 'comp_perf' as global variable
    global perf_score


    # append the results to the dataframe 'score_card'
    # 'ignore_index = True' do not consider the index labels
    perf_score = perf_score.append({'Model'       : name,
                                    'Accuracy'    : per_measures(model,test,pred)[0],
                                    'Recall'      : per_measures(model,test,pred)[1],
                                    'Precision'   : per_measures(model,test,pred)[2],
                                    'F1 Score'    : per_measures(model,test,pred)[3],
                                    'reliability' : per_measures(model,test,pred)[4]

                                     },
                                   ignore_index = True)

## SPLITTING THE DATA

In [ ]:
x=df4.drop('Survived', axis=1)
y=df4['Survived']

In [ ]:
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2, random_state=10)

In [ ]:
xtrain.shape,xtest.shape,ytrain.shape,ytest.shape

# Logistic regression -- base model

In [ ]:
# call the model
lgt=LogisticRegression(random_state=10)

In [ ]:
# fit the model
lgt.fit(xtrain,ytrain)

In [ ]:
# get the predictions 
y_pred=lgt.predict(xtest)

In [ ]:
# get the predicted probabilities 
y_pred_prop = lgt.predict_proba(xtest)[:,1]

In [ ]:
# update performance 
update_performance (name='logistic_base', model=lgt,test=ytest,pred=y_pred)

In [ ]:
perf_score

# KNN model - base model

In [ ]:
knn = KNeighborsClassifier()

In [ ]:
knn.fit(xtrain,ytrain)

In [ ]:
xtest_array = xtest.values if isinstance(xtest, pd.DataFrame) else xtest
y_pred = knn.predict(xtest_array)

In [ ]:
update_performance(name = 'KNN-Base', model = knn,test=ytest,pred=y_pred)
perf_score

# Decision Tree

In [ ]:
dt = DecisionTreeClassifier(random_state=10)
dt.fit(xtrain,ytrain)
ypred_dt = dt.predict(xtest)

In [ ]:
update_performance(name = 'Decision Tree-Gini', model = dt,test=ytest,
                   pred=ypred_dt)
perf_score

In [ ]:
dt_e = DecisionTreeClassifier(criterion='entropy',random_state=10)
dt_e.fit(xtrain,ytrain)
ypred_dt_e = dt_e.predict(xtest)

In [ ]:
update_performance(name = 'Decision Tree-Entropy', model = dt_e,test=ytest,pred=ypred_dt_e)
perf_score

# DecisionTree-TUNED

In [ ]:
tuned_paramaters = [{'criterion': ['entropy','gini'],
                     'max_depth': [3,5,7,9,11],
                     'max_features': ["sqrt", "log2"], 
                     'min_samples_split': [2,5,8],
                    'min_samples_leaf': [1,5,9], 
                     'max_leaf_nodes': [5,8]}]

In [ ]:
dt = DecisionTreeClassifier(random_state = 10)
tree_grid = GridSearchCV(estimator = dt, 
                         param_grid = tuned_paramaters,
                         cv = 5)

In [ ]:
tree_grid_model = tree_grid.fit(xtrain, ytrain)
tree_grid_model.best_params_

In [ ]:
dt_grid_model = DecisionTreeClassifier(criterion = 'entropy',
                                  max_depth = 5,
                                  max_features = 'log2',
                                  max_leaf_nodes = 8,
                                  min_samples_leaf = 9,
                                  min_samples_split = 2)

In [ ]:
dt_grid_model= dt_grid_model.fit(xtrain,ytrain)

In [ ]:
ypred_dt_tp = dt_grid_model.predict(xtest)

In [ ]:
update_performance(name = 'Decision Tree-tuned_entropy', model = dt_grid_model,test=ytest,pred=ypred_dt_tp)
perf_score

# Gaussian Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB

In [ ]:
gnb = GaussianNB()
gnb.fit(xtrain,ytrain)
ypred_gnb = gnb.predict(xtest)

In [ ]:
update_performance(name = 'Gaussian NB', 
                   model = gnb,
                   test=ytest,
                   pred=ypred_gnb)
perf_score

In [ ]:
bnb = BernoulliNB()
bnb.fit(xtrain,ytrain)

ypred_bnb = bnb.predict(xtest)

In [ ]:
update_performance(name = 'Bernoulli NB', model = bnb,test=ytest,pred=ypred_bnb)

# print the dataframe
perf_score

In [ ]:
mnb = MultinomialNB()

mnb.fit(xtrain,ytrain)


ypred_mnb = mnb.predict(xtest)

In [ ]:
update_performance(name = 'Multinomial NB', model = mnb,test=ytest,pred=ypred_mnb)

# print the dataframe
perf_score

# Random Forest

In [ ]:
rf = RandomForestClassifier(random_state=10)
rf.fit(xtrain,ytrain)
ypred_rf = rf.predict(xtest)

In [ ]:
update_performance(name = 'Random Forest', 
                   model = rf,
                   test=ytest,
                   pred=ypred_rf)
perf_score

###### OOB SAMPLE -PERFORMANCE

In [ ]:
rf = RandomForestClassifier(oob_score=True,random_state=10)

rf.fit(xtrain,ytrain)

ypred_rf = rf.predict(xtest)



In [ ]:
params =             [{'criterion': ['entropy', 'gini'],
                     'n_estimators': [100],
                     'max_depth': [10,20],
                     'max_features': ['sqrt', 'log2'],
                    'min_samples_split': [2, 8],
                     'min_samples_leaf': [5, 9],
                     'max_leaf_nodes': [8, 11]}]

In [ ]:
rf =RandomForestClassifier(random_state=10)

rf_cv = GridSearchCV(rf,params,cv=5,scoring='accuracy')

rf_cv.fit(xtrain,ytrain)

rf_cv.best_params_

In [ ]:

rf_model = RandomForestClassifier(criterion ='gini' ,
                                  n_estimators =100,
                                  max_depth = 10 ,
                                  max_features = 'sqrt',
                                  max_leaf_nodes =11,
                                  min_samples_leaf =5 ,
                                  min_samples_split = 2,
                                  )


rf_model.fit(xtrain,ytrain)

ypred_rf_tp = rf_model.predict(xtest)


In [ ]:
update_performance(name = 'Random Forest-Tunned', model = rf_model,test=ytest,pred=ypred_rf_tp)

# print the dataframe
perf_score

# Bagging classifier

In [ ]:
dt=DecisionTreeClassifier(random_state=10)
bc=BaggingClassifier(dt)
bc.fit(xtrain,ytrain)
ypred_bc = bc.predict(xtest)


In [ ]:
update_performance(name = 'Bagging Classifier-dt', 
                   model = bc,
                   test=ytest,
                   pred=ypred_bc)
perf_score

In [ ]:
knn=KNeighborsClassifier()

bag_knn=BaggingClassifier(knn) 
bag_knn.fit(xtrain,ytrain)

ypred_bag_knn = bag_knn.predict(xtest)

In [ ]:
update_performance(name = 'Bagging Classifier-knn', model = bag_knn,test=ytest,pred=ypred_bag_knn)

# print the dataframe
perf_score

In [ ]:
logr=LogisticRegression()

bag_logr=BaggingClassifier(logr) 
bag_logr.fit(xtrain,ytrain)

ypred_bag_logr = bag_logr.predict(xtest)



In [ ]:
update_performance(name = 'Bagging Classifier-Log', model = bag_logr,test=ytest,pred=ypred_bag_logr)

# print the dataframe
perf_score

# AdaBoost

In [ ]:
abcl = AdaBoostClassifier(random_state=10)
abcl.fit(xtrain,ytrain)

ypred_abcl = abcl.predict(xtest)

In [ ]:
update_performance(name = 'AdaBoost-dt', 
                   model = abcl,
                   test=ytest,
                   pred=ypred_abcl)
perf_score

# Gradientboost

In [ ]:
gbcl = GradientBoostingClassifier(random_state=10)
gbcl.fit(xtrain,ytrain)
ypred_gbcl = gbcl.predict(xtest)

In [ ]:
update_performance(name = 'Gradient Boosting', 
                   model = gbcl,
                   test=ytest,
                   pred=ypred_gbcl)

# print the dataframe
perf_score

# XGBOOST

In [ ]:
xgb= XGBClassifier(random_state=10)
xgb.fit(xtrain,ytrain)

ypred_xgb= xgb.predict(xtest)

In [ ]:
update_performance(name = 'XGB', model = xgb,test=ytest,pred=ypred_xgb)

# print the dataframe
perf_score

# STACKING 

In [ ]:
lgr=LogisticRegression()
RF=RandomForestClassifier()
dt=DecisionTreeClassifier()

In [ ]:
base_learners=[('lr_model',lgr),('DT_model',dt)]

stack =StackingClassifier(estimators=base_learners)
stack.fit(xtrain,ytrain)
ypred_stack = stack.predict(xtest_array )

In [ ]:
update_performance(name = 'Stacking', model = stack,test=ytest,pred=ypred_stack)

# print the dataframe
perf_score

#  casting the ballot

In [ ]:
vote_soft =VotingClassifier(estimators=base_learners ,voting='soft')

vote_soft.fit(xtrain,ytrain)
ypred_vote = vote_soft.predict(xtest)

In [ ]:
update_performance(name = 'Voting', model = vote_soft,test=ytest,pred=ypred_vote)

# print the dataframe
perf_score

# A Synopsis of the Models 

In [ ]:
perf_score.sort_values(by='F1 Score', ascending=False)

The model with the best performance among the ones I've built is gradient boost.

## Opted model is Decision Tree-tuned_entropy

In [ ]:
dt_grid_model

In [ ]:
ypred_dt_tp =dt_grid_model.predict(xtest)

In [ ]:
feature_imp = pd.DataFrame()
feature_imp['Feature']=xtrain.columns
feature_imp['Importance']=dt_grid_model.feature_importances_

In [ ]:
plt.figure(figsize=(10,7))
feature_imp = feature_imp.sort_values('Importance',ascending=False)
sns.barplot(x='Importance', y='Feature', data=feature_imp)
plt.show()

In [ ]:
ConfusionMatrixDisplay.from_predictions(ytest,ypred_dt_tp)
plt.show()

In [ ]:
y_pred_prob = dt_grid_model.predict_proba(xtest)[:,1]
y_pred_prob

In [ ]:
plt.figure(figsize=(10,6))
# the roc_curve() returns the values for false positive rate, true positive rate and threshold
# pass the actual target values and predicted probabilities to the function
fpr, tpr, thresholds = roc_curve(ytest, y_pred_prob)

# plot the ROC curve
plt.plot(fpr, tpr)

# set limits for x and y axes
plt.xlim([-0.1, 1.1])
plt.ylim([-0.1, 1.1])

# plot the straight line showing worst prediction for the model
plt.plot([0, 1], [0, 1],'r--')

# add plot and axes labels
# set text size using 'fontsize'
plt.title('ROC curve - Titanic survival prediction, DT-tuned model', fontsize = 15)
plt.xlabel('False positive rate (1-Specificity)', fontsize = 15)
plt.ylabel('True positive rate (Sensitivity)', fontsize = 15)

# add the AUC score to the plot
# 'x' and 'y' gives position of the text
# 's' is the text 
# use round() to round-off the AUC score upto 4 digits
plt.text(x = 0.02, y = 0.9, s = ('AUC Score:', round(roc_auc_score(ytest, y_pred_prob),4)))
                               
# plot the grid
plt.grid(True)

In [ ]:
ypred_train_rf_tp = rf_model.predict(xtrain)
ypred_test_rf_tp = rf_model.predict(xtest)

In [ ]:
# getting the actual values and predicted values of train 
a=pd.DataFrame({'ACTUAL':ytrain, 'PREDICTED': ypred_train_rf_tp})

In [ ]:
# getting the actual values and predicted values of test
b=pd.DataFrame({'ACTUAL':ytest, 'PREDICTED': ypred_test_rf_tp})

In [ ]:
#concating actual and predicted of test and train
c=pd.concat([a,b])
c

In [ ]:
# merging these values to the oroginal data frame. 
final_titanic=df2.join(c)

In [ ]:
# created a new column in dataframe as correct prediction or incorrect prediction 
def prediction(row):
    if row['ACTUAL'] == row['PREDICTED']:
        return 'correctprediction'
    else:
        return 'incorrectprediction'

final_titanic['prediction'] = final_titanic.apply(prediction, axis=1)


In [ ]:
# checking prediction count 
final_titanic['prediction'].value_counts()

## Examining the project

In [ ]:
# checking the percentage of correct and incorrect prediction 
b=final_titanic['prediction'].value_counts()
plt.figure(figsize=(6,8))
plt.pie(b, autopct='%1.1f%%',
        startangle=140,colors=['skyblue','black'])
plt.legend(b.index)
plt.title('Pie Chart for predictions from the model')
plt.show()

In [ ]:
# Considering that 'data' is your DataFrame with the passenger class, gender, and survival data
# Within each passenger class, group by gender using the 'hue' option.
plt.figure(figsize=(8, 6))
sns.barplot(y='PREDICTED', x='Sex', hue='Survived', data=final_titanic, palette='bone')
plt.ylabel('Percentage Survived')
plt.title('Percentage of Men and Women Survived in Each Passenger Class based Sex')
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
sns.barplot(x='Pclass', y='PREDICTED', hue='Survived', data=final_titanic, palette='bone')
plt.ylabel('Percentage Survived')
plt.title('Percentage of age group Survived in Each Passenger Class based on predictions')
plt.show()

# Conclusion 

There were only 38% of the passengers that made it out of the disaster alive, with 26% being women and 12% being men. Both the number of male passengers and the number of deaths were high. 

Although the first class has a notable survival rate in comparison to other classes, it is still unclear if survival is dependent on pclass. However, there appears to be a correlation between the class of the people who survive and their fare (who may include nobility). 

According to the data, there was a higher survival percentage among female passengers in any class on the Titanic than among male passengers. 
It's hard to say for sure if they adhered to the criteria in order to save the people. 

##### Well that was quit interesting, thanks for joining in 